## Expanding the CIP netCDFfiles and adding data from nav files

Status 'Turn into script'
- imports OK
- nc_save_with_check OK
- get_safire_flightid OK
- floor_to_sec_res OK
- Standardized_cip_netcdf replaced by update_cip_nc OK

TODO: 
- separate paths and struct information for reuse
- pull the metadata definitions outside of this program
- create .py file

In [1]:
import xarray as xr # read netcdf-files
import glob # allows for wildcards in filemanagement
import numpy as np
import os #get a list of all directories/files
import re #regex

from datetime import date
from utils.func_nc import floor_to_sec_res
from utils.nc_utils import nc_save_with_check, standardize_cip_netcdf, update_cip_nc
from utils.flight_utils import get_safire_flightid
from utils.meta_utils import var_attrs_from_list, attrs_from_list, meta_from_data

In [2]:
# sample rate to process (current possibilities: 1,5,12 sek)
sample_rate = 5


# ACDD metadata items, empty ('') are not added by attrs_fromlist
# HR: Highly recommended, R=Recommended, S= Suggested
# File and variable depended metadata items are added as attributes in separate dictionaries
meta_proj = {'Conventions': ['CF-1.8','ACDD'], # ACDD:HR
             'keywords': ['Earth Science > Atmosphere > Clouds > Cloud Microphysics > Cloud Liquid Water/Ice'], #ACDD:HR
            'source': 'airborne observation', #ACDD:R
            'comment': 'Uppercase attributes: SODA2 generated attributes, lowercase attributes: ACDD attributes, created post-SODA2 processing', #ACDD:R
            'project': 'ISLAS', #ACDD:R
            'platform': 'SAFIRE-ATR42', #ACDD:S
            'acknowledgement':'', #ACDD:R
            'creator_email':'', #ACDD:R
            'creator_name':'', #ACDD:R
            'creator_url':'', #ACDD:R
            'geospatial_bounds':'', #ACDD:R 
            'geospatial_bounds_crs':'', #ACDD:R 
            'geospatial_bounds_vertical_crs':'', #ACDD:R
            'geospatial_vertical_positive':'up', #ACDD:R
            'history':'', #ACDD:R
            'id':'', #ACDD:R
            'institution':'The University of Bergen', #ACDD:R
            'license':'', #ACDD:R
            'naming_authority':'', #ACDD:R
            'processing_level':'', #ACDD:R 
            'source':'aircraft observation', #ACDD:R
            'standard_name_vocabulary':'CF-1.8', #ACDD:R
            'cdm_data_type':'', #ACDD:S
            'contributor_name':'', #ACDD:S
            'contributor_role':'', #ACDD:S
            'creator_institution':'', #ACDD:S
            'creator_type':'', #ACDD:S
            'date_metadata_modified':'', #ACDD:S''
            'date_modified':'', #ACDD:S
            'geospatial_lat_resolution':'', #ACDD:S
            'geospatial_lat_units':'degree_north', #ACDD:S
            'geospatial_lon_resolution':'', #ACDD:S
            'geospatial_lon_units':'degree_east', #ACDD:S
            'geospatial_vertical_resolution':'', #ACDD:S
            'geospatial_vertical_units':'EPSG:4979', #ACDD:S
            'instrument_vocabulary':'', #ACDD:S
            'keywords_vocabulary':'GCMD:GCMD Keywords', #ACDD:S
            'metadata_link':'', #ACDD:S
            'platform_vocabulary':'', #ACDD:S
            'product_version':'1', #ACDD:S
            'program':'', #ACDD:S
            'references':''} #ACDD:S

# Update this when ready to publish! 
meta_publish = {'publisher_institution':'', #ACDD:S
                'publisher_type':'', #ACDD:S
                'publisher_email':'', #ACDD:R
                'publisher_name':'', #ACDD:R
                'publisher_url':'', #ACDD:R
                'date_issued':'', #ACDD:S
                'date_metadata_modified':date.today().strftime("%Y-%m-%d") #ACDD:S'

}

In [ ]:
# -- Paths to datafiles
nav_main_path = '/home/ninalar/Documents/MC2/2022-islas/' # Local disk path to nav data:
cip_main_path = f'/home/ninalar/Documents/MC2/Results_2022-islas/{sample_rate}sAveraging/'# Local disk path to SODA processed CIP files
save_path = '/home/ninalar/Documents/MC2/Results_2022-islas/Processed/CIP_processed/' # Save file path

# structure of file names (for access)
file_struct = {'cip':'/*CIP.nc',
               'cdp':'/02CDP*.csv',
               'nav_tdyn':'/*_TDYN_*.nc',
               'nav_nav': '/*_NAV_*.nc'}

flights, safire_to_islas = get_safire_flightid(nav_main_path)

In [ ]:
# Process all flights in the folders
for flight in flights:
    print(flight)

    # get navigation filenames
    #nav_tdyn_file = glob.glob(nav_main_path + flight + file_struct['nav_tdyn']) 
    nav_nav_file = glob.glob(nav_main_path + flight + file_struct['nav_nav'])

    # ---- Get CDP and NAV data from flight
    if isinstance(safire_to_islas[flight],list):
        # if more than one islasid for the safireid
        
        # flights with more than one islasids and must be addressed separately:
        for islasid in safire_to_islas[flight]:
            cip_file = glob.glob(cip_main_path + flight + f'/{islasid}' + file_struct['cip']) 
            print(f'Reading: {cip_file[0]}, {nav_nav_file[0]} and {nav_nav_file[0]}')
            cip_updated_xds = update_cip_nc(cip_file[0], nav_nav_file[0], islasid) # add islasid and coordinates to xds

            # Data dependent metadata
            meta_data = meta_from_data(cip_updated_xds) 
            cip_updated_xds = attrs_from_list(cip_updated_xds, meta_data) 

            # Flight/file dependent metadata
            filename = f'CIP_update_{sample_rate}s_{islasid}.nc'
            meta_file = {'summary': f'ISLAS-2022, post-processed CIP observations from flight {flight}', # ACDD:HR
            'title':f'Post-processed {sample_rate}s CIP observations from flight {flight}. Processed with SODA2 and added coordinates from coincident nav dataset', # ACDD:HR
            'date_created':date.today().strftime("%Y-%m-%d"), #ACDD:R
            }
            cip_updated_xds = attrs_from_list(cip_updated_xds, meta_file) 

            # Campaign metadata
            cip_updated_xds = attrs_from_list(cip_updated_xds, meta_proj)
            
            # save to new netCDF
            savefile = save_path + filename
            nc_save_with_check(savefile, cip_updated_xds)
    else:
        cip_file = glob.glob(cip_main_path + flight + file_struct['cip'])
        print(f'Reading: {cip_file[0]}, {nav_nav_file[0]} and {nav_nav_file[0]}')
        islasid = safire_to_islas[flight]
        #cip_updated_xds = standardize_cip_netcdf(cip_file[0], nav_tdyn_file[0],nav_nav_file[0], islasid) # add islasid and coordinates to xds
        cip_updated_xds = update_cip_nc(cip_file[0], nav_nav_file[0], islasid) # add islasid and coordinates to xds

        # Data dependent metadata
        meta_data = meta_from_data(cip_updated_xds) 
        cip_updated_xds = attrs_from_list(cip_updated_xds, meta_data) 

        # Flight/file dependent metadata
        filename = f'CIP_update_{sample_rate}s_{islasid}.nc'
        meta_file = {'summary': f'ISLAS-2022, post-processed CIP observations from flight {flight}', # ACDD:HR
        'title':f'Post-processed {sample_rate}s CIP observations from flight {flight}. Processed with SODA2 and added coordinates from coincident nav dataset', # ACDD:HR
        'date_created':date.today().strftime("%Y-%m-%d"), #ACDD:R
        }
        cip_updated_xds = attrs_from_list(cip_updated_xds, meta_file) 

        # Campaign metadata
        cip_updated_xds = attrs_from_list(cip_updated_xds, meta_proj)
            

        #Save to new netCDF
        savefile = save_path + f'CIP_update_{sample_rate}s_{islasid}.nc'      
        nc_save_with_check(savefile, cip_updated_xds)

as220014
Reading: /home/ninalar/Documents/MC2/Results_2022-islas/5sAveraging/as220014/04032022_071513_CIP.nc, /home/ninalar/Documents/MC2/2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220014_L1_V1.nc and /home/ninalar/Documents/MC2/2022-islas/as220014/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220403_as220014_L1_V1.nc
Exiting...
as220009
Reading: /home/ninalar/Documents/MC2/Results_2022-islas/5sAveraging/as220009/03262022_080110_CIP.nc, /home/ninalar/Documents/MC2/2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220009_L1_V1.nc and /home/ninalar/Documents/MC2/2022-islas/as220009/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220009_L1_V1.nc
Exiting...
as220010
Reading: /home/ninalar/Documents/MC2/Results_2022-islas/5sAveraging/as220010/03262022_145034_CIP.nc, /home/ninalar/Documents/MC2/2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220010_L1_V1.nc and /home/ninalar/Documents/MC2/2022-islas/as220010/ISLAS_SAFIRE-ATR42_CORE_NAV_1HZ_20220326_as220010_L1_

## Checking results etc..


In [5]:
cip_xds = xr.open_dataset(cip_file[0]) # returns an xarray dataset
#nav_tdyn_xds = xr.open_dataset(nav_tdyn_file[0]) # returns an xarray dataset
nav_nav_xds = xr.open_dataset(nav_nav_file[0]) # the nav file containing pitch, roll etc

In [6]:
cip_xds

<xarray.Dataset> Size: 3MB
Dimensions:           (Time: 2296, Vector64: 64, Vector40: 40)
Dimensions without coordinates: Time, Vector64, Vector40
Data variables: (12/33)
    elapsed_time      (Time) datetime64[ns] 18kB ...
    base_time         datetime64[ns] 8B ...
    utc_time          (Time) float64 18kB ...
    TAS               (Time) float32 9kB ...
    MIDBINS           (Vector64) float32 256B ...
    ACTIVETIME        (Time) float32 9kB ...
    ...                ...
    MVD200            (Time) float32 9kB ...
    IWC200            (Time) float32 9kB ...
    LWC200            (Time) float32 9kB ...
    AREA200           (Time) float32 9kB ...
    MEAN_AREARATIO    (Vector64, Time) float32 588kB ...
    MEAN_ASPECTRATIO  (Vector64, Time) float32 588kB ...
Attributes: (12/51)
    Source:                SODA-2 OAP Processing Software
    DateCreated:           Mon Feb  5 11:23:59 2024
    FlightDate:            03/30/2022
    DateProcessed:         Mon Feb  5 11:22:29 2024
    TimeInterval:          13:48:49-17:00:04
    FN:                    /home/ninalar/Documents/MC2/2022-islas/as220012/mi...
    ...                    ...
    CUSTOMDOF:             0.0
    CLUSTERTHRESH:         0.0
    RAKEFIX:               0
    IGNOREDEADTIME:        0
    STRICTCOUNTER:         0
    ACTIVETIMEFROMMISSED:  0

In [7]:
cip_updated_xds

<xarray.Dataset> Size: 4MB
Dimensions:           (time: 2296, Vector64: 64, Vector40: 40)
Coordinates:
    lat               (time) float32 9kB 67.82 67.82 67.82 ... 67.69 67.69 67.7
    lon               (time) float32 9kB 20.33 20.33 20.33 ... 20.11 20.12 20.12
    alt               (time) float32 9kB 451.2 451.2 ... 1.163e+03 1.156e+03
  * time              (time) datetime64[ns] 18kB 2022-03-30T13:48:49 ... 2022...
Dimensions without coordinates: Vector64, Vector40
Data variables: (12/33)
    base_time         datetime64[ns] 8B ...
    utc_time          (time) float64 18kB ...
    TAS               (time) float32 9kB ...
    MIDBINS           (Vector64) float32 256B ...
    ACTIVETIME        (time) float32 9kB ...
    SA                (Vector64) float32 256B ...
    ...                ...
    IWC200            (time) float32 9kB ...
    LWC200            (time) float32 9kB ...
    AREA200           (time) float32 9kB ...
    MEAN_AREARATIO    (Vector64, time) float32 588kB ...
    MEAN_ASPECTRATIO  (Vector64, time) float32 588kB ...
    SV_CIP            (Vector64, time) float32 588kB 0.000276 0.000276 ... 0.0
Attributes: (12/78)
    Source:                        SODA-2 OAP Processing Software
    DateCreated:                   Mon Feb  5 11:23:59 2024
    FlightDate:                    03/30/2022
    DateProcessed:                 Mon Feb  5 11:22:29 2024
    TimeInterval:                  13:48:49-17:00:04
    FN:                            /home/ninalar/Documents/MC2/2022-islas/as2...
    ...                            ...
    standard_name_vocabulary:      CF-1.8
    geospatial_lat_units:          degree_north
    geospatial_lon_units:          degree_east
    geospatial_vertical_units:     EPSG:4979
    keywords_vocabulary:           GCMD:GCMD Keywords
    product_version:               1

In [7]:
nav_nav_xds

<xarray.Dataset> Size: 892kB
Dimensions:     (time: 13110, level: 2)
Coordinates:
  * time        (time) datetime64[ns] 105kB 2022-03-30T13:29:19.400128 ... 20...
    LATITUDE    (time) float32 52kB ...
    LONGITUDE   (time) float32 52kB ...
    ALTITUDE    (time) float32 52kB ...
Dimensions without coordinates: level
Data variables:
    trajectory  |S100 100B ...
    time_bnds   (time, level) datetime64[ns] 210kB ...
    COURSE      (time) float32 52kB ...
    THEAD       (time) float32 52kB ...
    ROLL        (time) float32 52kB ...
    PITCH       (time) float32 52kB ...
    VN          (time) float32 52kB ...
    VE          (time) float32 52kB ...
    VV          (time) float32 52kB ...
    GS          (time) float32 52kB ...
Attributes: (12/34)
    creator_name:              SAFIRE
    creator_email:             UAR2859.campagne@safire.fr
    flight_id:                 as220012
    processing_level:          L1
    featureType:               Trajectory
    flight_date:               2022-03-30
    ...                        ...
    time_coverage_end:         2022-03-30T17:07:48.400022Z
    time_coverage_resolution:  PT1.000000S
    geospatial_lon_min:        19.700518521700005
    geospatial_lon_max:        25.094404127399986
    geospatial_lat_min:        67.6377160501
    geospatial_lat_max:        72.5088103494

In [ ]:
cip_xds.close()
#nav_tdyn_xds.close()
nav_nav_xds.close()